# Use case examples
This notebook demonstrates different use-cases for QC Lab

## Running a default simulation

In [1]:
from qclab import Simulation
from qclab.dynamics import serial_driver, parallel_driver_multiprocessing
from qclab.models import SpinBoson
from qclab.algorithms import MeanField
import numpy as np

sim_settings = {"num_trajs":100,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.initial_state.wf_db = np.array([1.0, 0.0], dtype=complex)
data = serial_driver(sim)
data = parallel_driver_multiprocessing(sim)

100%|██████████| 10001/10001 [00:02<00:00, 3741.31it/s]


## Changing the model ingredients

In [2]:
from qclab import Simulation
from qclab.dynamics import serial_driver, parallel_driver_multiprocessing
from qclab.models import SpinBoson
from qclab.algorithms import MeanField
import numpy as np
from qclab.ingredients import init_classical_wigner_harmonic

sim_settings = {"num_trajs":100,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.initial_state.wf_db = np.array([1.0, 0.0], dtype=complex)
print(sim.model.ingredients[5])
# change the init_classical ingredient from boltzmann to wigner.
sim.model.ingredients.insert(0,('init_classical', init_classical_wigner_harmonic))
print(sim.model.ingredients)
print(sim.model.ingredients[5])
data = serial_driver(sim)
data = parallel_driver_multiprocessing(sim)

('init_classical', <function init_classical_boltzmann_harmonic at 0x76adb672dc60>)
[('init_classical', <function init_classical_wigner_harmonic at 0x76adb672dcf0>), ('h_q', <function h_q_two_level at 0x76adb672d900>), ('h_qc', <function h_qc_diagonal_linear at 0x76adb672da20>), ('h_c', <function h_c_harmonic at 0x76adb6717c70>), ('dh_qc_dzc', <function dh_qc_dzc_diagonal_linear at 0x76adb672dab0>), ('dh_c_dzc', <function dh_c_dzc_harmonic at 0x76adb672d7e0>), ('init_classical', <function init_classical_boltzmann_harmonic at 0x76adb672dc60>), ('hop', <function hop_harmonic at 0x76adb672db40>), ('_init_h_q', <function SpinBoson._init_h_q at 0x76adb672e560>), ('_init_h_qc', <function SpinBoson._init_h_qc at 0x76adb672e5f0>), ('_init_model', <function SpinBoson._init_model at 0x76adb672e710>), ('_init_h_c', <function SpinBoson._init_h_c at 0x76adb672e680>)]
('dh_c_dzc', <function dh_c_dzc_harmonic at 0x76adb672d7e0>)


100%|██████████| 10001/10001 [00:02<00:00, 3886.14it/s]


## Changing algorithm tasks

In [6]:
from qclab import Simulation
from qclab.dynamics import serial_driver, parallel_driver_multiprocessing
from qclab.models import SpinBoson
from qclab.algorithms import MeanField
import numpy as np

sim_settings = {"num_trajs":100,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.initial_state.wf_db = np.array([1.0, 0.0], dtype=complex)
print(sim.algorithm.collect_recipe)
def collect_response(sim, state, parameters):
    # we can just calculate something simple like the response function
    if sim.t_ind == 0:
        state.wf_db_0 = np.copy(state.wf_db)
    state.response = np.sum(np.conj(state.wf_db_0) * state.wf_db, axis=-1)
    state.output_dict['response'] = state.response
    return state, parameters
sim.algorithm.collect_recipe.append(collect_response)
print(sim.algorithm.collect_recipe)
data = serial_driver(sim)
data = parallel_driver_multiprocessing(sim)

[<function update_t at 0x76adb672fac0>, <function update_dm_db_mf at 0x76adb6758e50>, functools.partial(<function update_quantum_energy at 0x76adb6759000>, wf='wf_db'), functools.partial(<function update_classical_energy at 0x76adb6758ee0>, z='z'), <function collect_t at 0x76adb6759240>, <function collect_dm_db at 0x76adb67592d0>, <function collect_classical_energy at 0x76adb6759360>, <function collect_quantum_energy at 0x76adb67593f0>]
[<function update_t at 0x76adb672fac0>, <function update_dm_db_mf at 0x76adb6758e50>, functools.partial(<function update_quantum_energy at 0x76adb6759000>, wf='wf_db'), functools.partial(<function update_classical_energy at 0x76adb6758ee0>, z='z'), <function collect_t at 0x76adb6759240>, <function collect_dm_db at 0x76adb67592d0>, <function collect_classical_energy at 0x76adb6759360>, <function collect_quantum_energy at 0x76adb67593f0>, <function collect_response at 0x76ada9ae28c0>]


100%|██████████| 10001/10001 [00:02<00:00, 3511.79it/s]


## User supplied seeds



In [7]:
np.random.seed(1234)

my_seeds = np.unique(np.random.randint(0,10000, size=100))

from qclab import Simulation
from qclab.dynamics import serial_driver, parallel_driver_multiprocessing
from qclab.models import SpinBoson
from qclab.algorithms import MeanField
import numpy as np

sim_settings = {"num_trajs":100,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.initial_state.wf_db = np.array([1.0, 0.0], dtype=complex)
data_serial = serial_driver(sim, seeds=my_seeds)

assert np.all(data_serial.data_dict['seed'] == my_seeds)
print('validated serial_driver with seeds')

data_parallel = parallel_driver_multiprocessing(sim, seeds=my_seeds)

assert np.all(data_parallel.data_dict['seed'] == my_seeds)
print('validated parallel_driver_multiprocessing with seeds')

for key, val in data_serial.data_dict.items():
    if key in data_parallel.data_dict:
        assert np.all(val == data_parallel.data_dict[key]), f"Mismatch in {key} between serial and parallel drivers"
print('validated data consistency between serial and parallel drivers')


100%|██████████| 10001/10001 [00:01<00:00, 5910.71it/s]

validated serial_driver with seeds



100%|██████████| 10001/10001 [00:02<00:00, 3885.26it/s]


validated parallel_driver_multiprocessing with seeds
validated data consistency between serial and parallel drivers


## User supplied data object

In [8]:
## Serial
from qclab import Simulation
from qclab.dynamics import serial_driver, parallel_driver_multiprocessing
from qclab.models import SpinBoson
from qclab.algorithms import MeanField
import numpy as np

sim_settings = {"num_trajs":100,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.initial_state.wf_db = np.array([1.0, 0.0], dtype=complex)
data_1 = serial_driver(sim)
data_2 = serial_driver(sim, data=data_1)

sim_settings = {"num_trajs":200,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.initial_state.wf_db = np.array([1.0, 0.0], dtype=complex)
data_3 = serial_driver(sim)

for key, val in data_3.data_dict.items():
    if key in data_2.data_dict:
        assert np.all(val == data_2.data_dict[key]), f"Mismatch in {key}"
print('validated data consistency between supplied data and new simulation')

  0%|          | 0/10001 [00:00<?, ?it/s]

100%|██████████| 10001/10001 [00:01<00:00, 5732.04it/s]

validated data consistency between supplied data and new simulation


In [9]:
## Parallel
from qclab import Simulation
from qclab.dynamics import serial_driver, parallel_driver_multiprocessing
from qclab.models import SpinBoson
from qclab.algorithms import MeanField
import numpy as np

sim_settings = {"num_trajs":100,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.initial_state.wf_db = np.array([1.0, 0.0], dtype=complex)
data_1 = parallel_driver_multiprocessing(sim)
data_2 = parallel_driver_multiprocessing(sim, data=data_1)
print(len(data_2.data_dict['seed']))
sim_settings = {"num_trajs":200,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.initial_state.wf_db = np.array([1.0, 0.0], dtype=complex)
data_3 = parallel_driver_multiprocessing(sim)

for key, val in data_3.data_dict.items():
    if key in data_2.data_dict:
        print(f"Validating {key}")
        assert np.all(val == data_2.data_dict[key]), f"Mismatch in {key}"
print('validated data consistency between supplied data and new simulation')

100%|██████████| 10001/10001 [00:02<00:00, 4159.42it/s]


200


100%|██████████| 10001/10001 [00:04<00:00, 2103.76it/s]


Validating seed
Validating norm_factor
Validating t
Validating dm_db
Validating classical_energy
Validating quantum_energy
validated data consistency between supplied data and new simulation
